Just recomputing these because I want to report percentile changes and can't find the relevant numbers.

In [1]:
import xarray as xr
import numpy as np
import dask.array
import cosima_cookbook as cc
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/home/156/rm2389/Honours_Thesis/python_functions/')
from spatial_averaging import month_weights
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
from dask.distributed import Client
client = Client('tcp://localhost:8786')
client

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 6 Cores: 6 Memory: 18.00 GB


In [2]:
db = '../../freshwater_experiments.db' # database path
session = cc.database.create_session(db)
control='01deg_jra55v13_ryf8485_spinup6'
rcp45='01deg_jra55v13_ryf8485_freshRCP45'
rcp85='01deg_jra55v13_ryf8485_freshRCP85'

### Thickness

In [3]:
control_ice_thickness = cc.querying.getvar(control, 'hi_m', session)
rcp45_ice_thickness = cc.querying.getvar(rcp45, 'hi_m', session)
rcp85_ice_thickness = cc.querying.getvar(rcp85, 'hi_m', session)

In [4]:
def replace_ice_coords(ice, tgrid): ## note - dont use for arctic cause of tripole inaccuracies
    ice.coords['nj'] = tgrid.yt_ocean.values
    ice.coords['ni'] = tgrid.xt_ocean.values
    ice = ice.rename(({'nj':'yt_ocean', 'ni':'xt_ocean'}))
    return ice

In [25]:
dyt = cc.querying.getvar(control, 'dyt', session, ncfile='ocean_grid.nc', n=1)
dxt = cc.querying.getvar(control, 'dxt', session, ncfile='ocean_grid.nc', n=1)

In [6]:
control_ice_thickness = replace_ice_coords(control_ice_thickness, dyt)
rcp45_ice_thickness = replace_ice_coords(rcp45_ice_thickness, dyt)
rcp85_ice_thickness = replace_ice_coords(rcp85_ice_thickness, dyt)

In [9]:
timeslice = slice('1937-06','1947-05')
latslice = slice(-90,-59)
control_ice_thickness = control_ice_thickness.sel(time = timeslice, yt_ocean = latslice)
rcp45_ice_thickness = rcp45_ice_thickness.sel(time = timeslice, yt_ocean = latslice)
rcp85_ice_thickness = rcp85_ice_thickness.sel(time = timeslice, yt_ocean = latslice)

In [11]:
def shelf_mask_isobath(var):
    contour_file = np.load('/home/156/rm2389/Antarctic_slope_contour_1000m.npz')
    shelf_mask = contour_file['contour_masked_above']
    yt_ocean = contour_file['yt_ocean']
    xt_ocean = contour_file['xt_ocean']
    # in this file the points along the isobath are given a positive value, the points outside (northwards) of the isobatj
    # are given a value of -100 and all the points on the continental shelf have a value of 0 so we mask for the 0 values 
    shelf_mask[np.where(shelf_mask!=0)] = np.nan
    shelf_mask = shelf_mask+1
    shelf_map = np.nan_to_num(shelf_mask)
    shelf_mask  = xr.DataArray(shelf_mask,coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)] )
    shelf_map  = xr.DataArray(shelf_map,coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)] )
    # then we want to multiply the variable with the mask so we need to account for the shape of the mask. The mask 
    # uses a northern cutoff of 59S.
    masked_var = var.sel(yt_ocean = slice(-90,-59.03)) * shelf_mask
    return masked_var, shelf_map

In [12]:
control_ice_thickness, shelf_mask = shelf_mask_isobath(control_ice_thickness)
rcp45_ice_thickness, shelf_mask = shelf_mask_isobath(rcp45_ice_thickness)
rcp85_ice_thickness, shelf_mask = shelf_mask_isobath(rcp85_ice_thickness)
dyt, shelf_mask = shelf_mask_isobath(dyt)
dxt, shelf_mask = shelf_mask_isobath(dxt)

In [14]:
##time mean
time = control_ice_thickness.time.values
weights = xr.DataArray(month_weights(start_month=6, end_month=5, n_years=10), coords = [time], dims = 'time')
control_hi_time_mean = (control_ice_thickness*weights).mean(dim = 'time')
rcp45_hi_time_mean = (rcp45_ice_thickness*weights).mean(dim = 'time')
rcp85_hi_time_mean = (rcp85_ice_thickness*weights).mean(dim = 'time')

In [15]:
def surface_averaged(var, dxt, dyt):
    var_scaled = var  * dxt * dyt
    var_scaled_sum = var_scaled.sum(dim = 'xt_ocean').sum(dim = 'yt_ocean')
    area = dxt * dyt 
    area_sum = area.sum(dim = 'xt_ocean').sum(dim = 'yt_ocean')
    var_area_mean = var_scaled_sum /area_sum;
    return var_area_mean

In [16]:
##full shelf
print('Values in m, averaged over 10 year experimental period')
shelf_control_hi_time_mean = surface_averaged(control_hi_time_mean, dxt, dyt).load()
print('Full shelf ice thickness, control:',shelf_control_hi_time_mean.values)
shelf_rcp45_hi_time_mean = surface_averaged(rcp45_hi_time_mean, dxt, dyt).load()
print('Full shelf ice thickness, rcp45:',shelf_rcp45_hi_time_mean.values)
shelf_rcp85_hi_time_mean = surface_averaged(rcp85_hi_time_mean, dxt, dyt).load()
print('Full shelf ice thickness, rcp85:',shelf_rcp85_hi_time_mean.values)

Values in m, averaged over 10 year experimental period
Full shelf ice thickness, control: 0.8994173184954433
Full shelf ice thickness, rcp45: 0.9053248232099077
Full shelf ice thickness, rcp85: 0.9170622627058942


In [17]:
print('Values in % increase, relative to the control, averaged over 10 year experimental period')
shelf_rcp45_hi_pi = (shelf_rcp45_hi_time_mean-shelf_control_hi_time_mean)/shelf_control_hi_time_mean *100
print('Full shelf ice thickness, rcp45:',shelf_rcp45_hi_pi.values)
shelf_rcp85_hi_pi = (shelf_rcp85_hi_time_mean-shelf_control_hi_time_mean)/shelf_control_hi_time_mean *100
print('Full shelf ice thickness, rcp45:',shelf_rcp85_hi_pi.values)

Values in % increase, relative to the control, averaged over 10 year experimental period
Full shelf ice thickness, rcp45: 0.6568146502167191
Full shelf ice thickness, rcp45: 1.9618194855273208


In [18]:
##ross sea
## lon bounds are -198 -157
lonslice = slice(-198,-157)
print('Values in m, averaged over 10 year experimental period')
ross_control_hi_time_mean = surface_averaged(control_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('ross shelf ice thickness, control:',ross_control_hi_time_mean.values)
ross_rcp45_hi_time_mean = surface_averaged(rcp45_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('ross shelf ice thickness, rcp45:',ross_rcp45_hi_time_mean.values)
ross_rcp85_hi_time_mean = surface_averaged(rcp85_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('ross shelf ice thickness, rcp85:',ross_rcp85_hi_time_mean.values)

Values in m, averaged over 10 year experimental period
ross shelf ice thickness, control: 0.7477423743593532
ross shelf ice thickness, rcp45: 0.8120933580874587
ross shelf ice thickness, rcp85: 0.9444720039035095


In [21]:
print('Values in % increase, relative to the control, averaged over 10 year experimental period')
ross_rcp45_hi_pi = (ross_rcp45_hi_time_mean-ross_control_hi_time_mean)/ross_control_hi_time_mean *100
print('ross shelf ice thickness, rcp45:',ross_rcp45_hi_pi.values)
ross_rcp85_hi_pi = (ross_rcp85_hi_time_mean-ross_control_hi_time_mean)/ross_control_hi_time_mean *100
print('ross shelf ice thickness, rcp45:',ross_rcp85_hi_pi.values)

Values in % increase, relative to the control, averaged over 10 year experimental period
ross shelf ice thickness, rcp45: 8.606036776134275
ross shelf ice thickness, rcp45: 26.309814220801552


In [19]:
##west antarctica
## lon bounds are -157, -63
lonslice = slice(-157,-63)
print('Values in m, averaged over 10 year experimental period')
wa_control_hi_time_mean = surface_averaged(control_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('west antarctica shelf ice thickness, control:',wa_control_hi_time_mean.values)
wa_rcp45_hi_time_mean = surface_averaged(rcp45_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('west antarctica shelf ice thickness, rcp45:',wa_rcp45_hi_time_mean.values)
wa_rcp85_hi_time_mean = surface_averaged(rcp85_hi_time_mean.sel(xt_ocean = lonslice), dxt.sel(xt_ocean = lonslice), dyt.sel(xt_ocean = lonslice)).load()
print('west antarctica shelf ice thickness, rcp85:',wa_rcp85_hi_time_mean.values)

Values in m, averaged over 10 year experimental period
west antarctica shelf ice thickness, control: 1.3188331621622014
west antarctica shelf ice thickness, rcp45: 1.2422222852996396
west antarctica shelf ice thickness, rcp85: 1.0728745491243523


In [22]:
print('Values in % increase, relative to the control, averaged over 10 year experimental period')
wa_rcp45_hi_pi = (wa_rcp45_hi_time_mean-wa_control_hi_time_mean)/wa_control_hi_time_mean *100
print('wa shelf ice thickness, rcp45:',wa_rcp45_hi_pi.values)
wa_rcp85_hi_pi = (wa_rcp85_hi_time_mean-wa_control_hi_time_mean)/wa_control_hi_time_mean *100
print('wa shelf ice thickness, rcp45:',wa_rcp85_hi_pi.values)

Values in % increase, relative to the control, averaged over 10 year experimental period
wa shelf ice thickness, rcp45: -5.808989268738109
wa shelf ice thickness, rcp45: -18.649714011938


### Congelation + Frazil (formation)

In [23]:
control_ice_cong = cc.querying.getvar(control, 'congel_m', session)
rcp45_ice_cong = cc.querying.getvar(rcp45, 'congel_m', session)
rcp85_ice_cong = cc.querying.getvar(rcp85, 'congel_m', session)
control_ice_frazil = cc.querying.getvar(control, 'frazil_m', session)
rcp45_ice_frazil = cc.querying.getvar(rcp45, 'frazil_m', session)
rcp85_ice_frazil = cc.querying.getvar(rcp85, 'frazil_m', session)

In [26]:
control_ice_cong = replace_ice_coords(control_ice_cong, dyt)
rcp45_ice_cong = replace_ice_coords(rcp45_ice_cong, dyt)
rcp85_ice_cong = replace_ice_coords(rcp85_ice_cong, dyt)
control_ice_frazil = replace_ice_coords(control_ice_frazil, dyt)
rcp45_ice_frazil = replace_ice_coords(rcp45_ice_frazil, dyt)
rcp85_ice_frazil = replace_ice_coords(rcp85_ice_frazil, dyt)

In [28]:
timeslice = slice('1937-06','1947-05')
latslice = slice(-90,-59)
control_ice_cong = control_ice_cong.sel(time = timeslice, yt_ocean = latslice)
rcp45_ice_cong = rcp45_ice_cong.sel(time = timeslice, yt_ocean = latslice)
rcp85_ice_cong = rcp85_ice_cong.sel(time = timeslice, yt_ocean = latslice)
control_ice_frazil = control_ice_frazil.sel(time = timeslice, yt_ocean = latslice)
rcp45_ice_frazil = rcp45_ice_frazil.sel(time = timeslice, yt_ocean = latslice)
rcp85_ice_frazil = rcp85_ice_frazil.sel(time = timeslice, yt_ocean = latslice)

In [30]:
control_ice_cong, shelf_mask = shelf_mask_isobath(control_ice_cong)
rcp45_ice_cong, shelf_mask = shelf_mask_isobath(rcp45_ice_cong)
rcp85_ice_cong, shelf_mask = shelf_mask_isobath(rcp85_ice_cong)
control_ice_frazil, shelf_mask = shelf_mask_isobath(control_ice_frazil)
rcp45_ice_frazil, shelf_mask = shelf_mask_isobath(rcp45_ice_frazil)
rcp85_ice_frazil, shelf_mask = shelf_mask_isobath(rcp85_ice_frazil)
dyt, shelf_mask = shelf_mask_isobath(dyt)
dxt, shelf_mask = shelf_mask_isobath(dxt)

In [31]:
##time mean
time = control_ice_cong.time.values
weights = xr.DataArray(month_weights(start_month=6, end_month=5, n_years=10), coords = [time], dims = 'time')
control_cong_time_mean = (control_ice_cong*weights).mean(dim = 'time')
rcp45_cong_time_mean = (rcp45_ice_cong*weights).mean(dim = 'time')
rcp85_cong_time_mean = (rcp85_ice_cong*weights).mean(dim = 'time')
control_frazil_time_mean = (control_ice_frazil*weights).mean(dim = 'time')
rcp45_frazil_time_mean = (rcp45_ice_frazil*weights).mean(dim = 'time')
rcp85_frazil_time_mean = (rcp85_ice_frazil*weights).mean(dim = 'time')

In [42]:
control_cong_time_mean = control_cong_time_mean.reset_coords(drop=True)
rcp45_cong_time_mean = rcp45_cong_time_mean.reset_coords(drop=True)
rcp85_cong_time_mean = rcp85_cong_time_mean.reset_coords(drop=True)
control_frazil_time_mean = control_frazil_time_mean.reset_coords(drop=True)
rcp45_frazil_time_mean = rcp45_frazil_time_mean.reset_coords(drop=True)
rcp85_frazil_time_mean = rcp85_frazil_time_mean.reset_coords(drop=True)

In [44]:
control_prod_time_mean = control_cong_time_mean + control_frazil_time_mean
rcp45_prod_time_mean = rcp45_cong_time_mean + rcp45_frazil_time_mean
rcp85_prod_time_mean = rcp85_cong_time_mean + rcp85_frazil_time_mean

In [46]:
##full shelf
print('Values in cm/day, averaged over 10 year experimental period')
shelf_control_prod_time_mean = surface_averaged(control_prod_time_mean, dxt, dyt).load()
print('Full shelf ice production, control:',shelf_control_prod_time_mean.values)
shelf_rcp45_prod_time_mean = surface_averaged(rcp45_prod_time_mean, dxt, dyt).load()
print('Full shelf ice production, rcp45:',shelf_rcp45_prod_time_mean.values)
shelf_rcp85_prod_time_mean = surface_averaged(rcp85_prod_time_mean, dxt, dyt).load()
print('Full shelf ice production, rcp85:',shelf_rcp85_prod_time_mean.values)

Values in cm/day, averaged over 10 year experimental period
Full shelf ice thickness, control: 0.9988576759633597
Full shelf ice thickness, rcp45: 1.0658756283796853
Full shelf ice thickness, rcp85: 1.1775715594495861


In [47]:
print('Values in % increase, relative to the control, averaged over 10 year experimental period')
shelf_rcp45_prod_pi = (shelf_rcp45_prod_time_mean-shelf_control_prod_time_mean)/shelf_control_prod_time_mean *100
print('Full shelf ice production, rcp45:',shelf_rcp45_hi_pi.values)
shelf_rcp85_prod_pi = (shelf_rcp85_prod_time_mean-shelf_control_prod_time_mean)/shelf_control_prod_time_mean *100
print('Full shelf ice production, rcp45:',shelf_rcp85_prod_pi.values)

Values in % increase, relative to the control, averaged over 10 year experimental period
Full shelf ice production, rcp45: 0.6568146502167191
Full shelf ice production, rcp45: 17.89182661222118
